# Plot Y Chromosome Gene Blood Expression From AMP-PD Data with Different Y Chromosome Haplogroups
- **Author(s)** - Frank Grenn
- **Date Started** - August  2021
- **Quick Description:** Get AMP-PD expression data, plot with haplogroup and case/control status

In [ ]:

import os

import pandas as pd
import mygene

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
WRKDIR = "$PATH"


## AMP-PD Gene Expression Data

In [ ]:
amp_counts = pd.read_csv(f"{WRKDIR}/chrY/expression/amppd_chrY_featureCounts.csv")
amp_counts = amp_counts.set_index("Geneid")
print(amp_counts.shape)
print(amp_counts.iloc[0:5,0:5])

## Sample Data


In [ ]:
meta = pd.read_csv(f"{WRKDIR}/chrY/output_male_hemizygous_only_het_filter_run/chrY_meta.csv")

print(meta.shape)
print(meta.head())

## Get list of genes to plot

In [ ]:
results = pd.read_csv(f"{WRKDIR}/chrY/expression/amppd_haplogroup_G_diff_exp_edgeR_results_gene_symbol.csv")
print(results.shape)
print(results.head())

In [ ]:
genes_to_plot = results.loc[(results.FDR<0.05) & (results.logFC>2),].iloc[:,0].tolist()
print(len(genes_to_plot))

In [ ]:
print(genes_to_plot)

## Plot for a multiple genes

In [ ]:
#rows and cols in plot grid. Need to be able to fit everything in ct list above
rows = 2
cols = 2

#haplogroups to include in the plots
haplos = ['R','I','J','E','G']

In [ ]:


gene_count = 0;
subplot_count = 0
fig = plt.figure(figsize=(12, 8), dpi=80)
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i in range(rows):
    for j in range(cols):

        gene = genes_to_plot[gene_count]
        print(gene)
        
        full_gene = [g for g in amp_counts.index if gene in g][0]
        print(full_gene)

        title = f"{results.loc[results.gene_no_ver==gene,'gene_symbol'].iloc[0]} ({full_gene})"
        print(title)
        
        gene_counts = amp_counts[[gene in i for i in amp_counts.index]].transpose()
        print(gene_counts.shape)
        
        meta_quant = pd.merge(left = meta, left_on= 'fid',right = gene_counts, right_index=True)
        meta_quant = meta_quant.rename(columns = {full_gene:'counts'})
        print(meta_quant.shape)
        print(meta_quant.head())

        

        #add subplot
        ax = fig.add_subplot(rows,cols,subplot_count+1)

        #plot
        sns.set()


        sns_plot = sns.violinplot(palette="tab10",x='yhaplo_haplo_major', y='counts', data=meta_quant,inner = "box",order=sorted(haplos),hue='pheno',split=False)
        sns_plot = sns.stripplot(palette="tab10",linewidth=1,x='yhaplo_haplo_major', y='counts', data=meta_quant,color=".4",order=sorted(haplos),hue='pheno',dodge=True)
        #sns_plot = sns.boxplot(palette="tab10",x="yhaplo_haplo_major", y="TPM",color=".4",data=df, order=sorted(haplogroups),hue="pheno" )

        plt.xlabel(f"Y Chromosome Haplogroup")
        plt.ylabel(f"Counts")
        plt.title(f"{title}")

        handles, labels = sns_plot.get_legend_handles_labels()

        labels = ['control','case','control','case']

        # When creating the legend, only use the first two elements
        # to effectively remove the last two.
        l = plt.legend(handles[0:2], labels[0:2], bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)

        subplot_count = subplot_count + 1 

    

        gene_count = gene_count + 1
        if(gene_count >= len(genes_to_plot)):
            break;
            
    if(gene_count >= len(genes_to_plot)):
        break;



fig.suptitle(f"Genes Highly Expressed in AMP-PD Samples with Major Haplogroup G")
plt.show()
sns_plot.get_figure().savefig(f"{WRKDIR}/chrY/expression/haplo_g_amppd_counts_violin_multiplot.png")   




In [ ]:
amp_counts.loc["ENSG00000184895.7",]

In [ ]:
for g in genes_to_plot:
    print(g)

    print(amp_counts[[g in i for i in amp_counts.index]].shape)
    print(amp_counts[[g in i for i in amp_counts.index]].transpose().iloc[0:5,0:5])

In [ ]:
genes = ["ENSG00000184895.7"]

In [ ]:
amp_counts.loc[gene,].to_frame()

In [ ]:
amp_counts.loc[gene,].describe()

In [ ]:
meta_quant = pd.merge(left = meta, left_on= 'fid',right = amp_counts.loc[gene,].to_frame(), right_index=True)
meta_quant = meta_quant.rename(columns = {gene:'counts'})
print(meta_quant.shape)
print(meta_quant.head())

In [ ]:
#check haplogroup counts to see which are worth plotting.
meta_quant.yhaplo_haplo_major.value_counts()

In [ ]:
haplos = ['R','I','J','E','G']

In [ ]:
def haplogroup_violin_plot(df, title, haplogroups, save_file = None, combine_all=True):
    

    #if we want a violin plot with all data combined, the duplicate the df, set the haplo col to 'all_haplogroups' and add to the main dataframe
    if (combine_all):
        combine_df = df.copy()
        #print(combine_df.yhaplo_haplo_major.value_counts())
        combine_df = combine_df[combine_df.yhaplo_haplo_major.isin(haplogroups)]
        #print(combine_df.yhaplo_haplo_major.value_counts())
        combine_df.yhaplo_haplo_major = 'All Haplogroups'
        df = df.append(combine_df, ignore_index=True)
        haplogroups = haplogroups + ['All Haplogroups']
    
    #plot
    plt.figure(figsize=(7, 5))
    sns.set()

    
    sns_plot = sns.violinplot(palette="tab10",x='yhaplo_haplo_major', y='counts', data=df,inner = "box",order=sorted(haplogroups),hue='pheno',split=False)
    sns_plot = sns.stripplot(palette="tab10",linewidth=1,x='yhaplo_haplo_major', y='counts', data=df,color=".4",order=sorted(haplogroups),hue='pheno',dodge=True)
    #sns_plot = sns.boxplot(palette="tab10",x="yhaplo_haplo_major", y="TPM",color=".4",data=df, order=sorted(haplogroups),hue="pheno" )

    plt.xlabel(f"Y Chromosome Haplogroup")
    plt.ylabel(f"Counts")
    plt.title(title)

    handles, labels = sns_plot.get_legend_handles_labels()

    labels = ['control','case','control','case']

    # When creating the legend, only use the first two elements
    # to effectively remove the last two.
    l = plt.legend(handles[0:2], labels[0:2], bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)


    plt.show()
    if (save_file is not None):
        sns_plot.get_figure().savefig(save_file,bbox_inches='tight')

In [ ]:
haplogroup_violin_plot(meta_quant,f"SRY ({gene}) Expression", haplos, save_file = f"{WRKDIR}/chrY/expression/amp_sry_counts_plot.png",combine_all=True)